In [3]:
from flask import Flask, render_template, redirect, jsonify
from sqlalchemy import create_engine
import pandas as pd
import json

import sqlalchemy as db

connection_string = 'postgres:Adgjmpt1@postresql.cfwwjixawhdl.us-east-2.rds.amazonaws.com:5432/RecipeGenerator'
engine = db.create_engine(f'postgresql://{connection_string}', echo=True)
connection = engine.connect()
metadata = db.MetaData()

2020-12-02 10:34:49,218 INFO sqlalchemy.engine.base.Engine select version()
2020-12-02 10:34:49,220 INFO sqlalchemy.engine.base.Engine {}
2020-12-02 10:34:49,290 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-12-02 10:34:49,291 INFO sqlalchemy.engine.base.Engine {}
2020-12-02 10:34:49,362 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-12-02 10:34:49,363 INFO sqlalchemy.engine.base.Engine {}
2020-12-02 10:34:49,399 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-12-02 10:34:49,399 INFO sqlalchemy.engine.base.Engine {}
2020-12-02 10:34:49,436 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-12-02 10:34:49,437 INFO sqlalchemy.engine.base.Engine {}


In [2]:
# Recipe
table = db.Table('recipe_list', metadata, autoload = True, autoload_with = engine)
query = db.select(
    [
        table
    ]
).where(table.columns.id == 1)

result = connection.execute(query).fetchall()
recipe_response = []
for ea in result:
    recipe_response.append(
        {
            "id": ea[0],
            "name": ea[1],
            'description': ea[2],
            'category': ea[3],
            'subcategory': ea[4],
            'prep_time': ea[5],
            'total_time': ea[6],
            'calories': ea[7],
            'hf_id': ea[8],
            'tip': ea[9],
            'image_url': ea[10]
        }
    )

2020-12-02 10:34:40,816 INFO sqlalchemy.engine.base.Engine 
            SELECT c.oid
            FROM pg_catalog.pg_class c
            LEFT JOIN pg_catalog.pg_namespace n ON n.oid = c.relnamespace
            WHERE (pg_catalog.pg_table_is_visible(c.oid))
            AND c.relname = %(table_name)s AND c.relkind in
            ('r', 'v', 'm', 'f', 'p')
        
2020-12-02 10:34:40,816 INFO sqlalchemy.engine.base.Engine {'table_name': 'recipe_list'}
2020-12-02 10:34:40,887 INFO sqlalchemy.engine.base.Engine 
            SELECT a.attname,
              pg_catalog.format_type(a.atttypid, a.atttypmod),
              (SELECT pg_catalog.pg_get_expr(d.adbin, d.adrelid)
                FROM pg_catalog.pg_attrdef d
               WHERE d.adrelid = a.attrelid AND d.adnum = a.attnum
               AND a.atthasdef)
              AS DEFAULT,
              a.attnotnull, a.attnum, a.attrelid as table_oid,
              pgd.description as comment,
              a.attgenerated as generated
            F

In [4]:
recipe_df

NameError: name 'recipe_df' is not defined

In [ ]:
# Ingredients
table = db.Table('ingredients', metadata, autoload = True, autoload_with = engine)
query = db.select(
    [
        table
    ]
).where(table.columns.recipe_id == 1)

result = connection.execute(query).fetchall()
ingredients_response = []
for ea in result:
    ingredients_response.append(
        {
            "id": ea[0],
            "recipe_id": ea[1],
            'ingredient_id': ea[2],
            'qty': ea[3],
            'unit': ea[4],
            'servings': ea[5]
        }
    )

In [ ]:
# Ingredients Lookup
table = db.Table('ingredients_list', metadata, autoload = True, autoload_with = engine)
query = db.select(
    [
        table
    ]
)

result = connection.execute(query).fetchall()
ingredients_list_response = []
for ea in result:
    ingredients_list_response.append(
        {
            "id": ea[0],
            "ingredient": ea[1],
            'category': ea[2],
            'subcategory': ea[3],
            'staple': ea[4]
        }
    )

In [ ]:
ingredients_df = pd.DataFrame(ingredients_response)
ingredients_list_df = pd.DataFrame(ingredients_list_response)

In [ ]:
ingr_df = ingredients_df.merge(ingredients_list_df, left_on='ingredient_id', right_on='id')

In [ ]:
ingr_df = ingr_df[['ingredient','qty','unit','servings']]
ingr_df

In [ ]:
res = ingr_df.to_json(orient="records")
parsed = json.loads(res)
parsed

In [ ]:
recipe_response[0]['ingredients'] = parsed

In [ ]:
recipe_response